# Track trading costs and commissions in your portfolio

The aim of this notebook is to show how LUSID can be used to track trading comissions in your portfolio. As a portfolio manager, you might want to track commissions and fees seperate from the trade cost itself. This seperation allows you to (a) track trading commissions against other portfolios, and (b) to measure your P&L and performance inclusive or exclusive of comissions. 

<br>

Consider the example below. Here we might want to track the "Trade Cost" seperate from the "Fees" and "Net money on trade". LUSID has functionality which allows us to easily store all these data points.


<img src="img/trade-comms/trade_cost.PNG" alt="Drawing" style="width: 800px;"/>

### Table of contents

1. [Setup client and scope](#setup)<br>
2. [Read in transactions data CSV file](#trd_csv_read)<br>
3. [Set up mapping for upserting instruments, portfolios, and transactions](#trd_map)<br>
4. [Set up the instrument master](#trd_im)<br>
5. [Create property definition for a sub-holding key to track commissions ](#prop_def)<br>
6. [Create the portfolio](#trd_port)<br>
7. [Upsert your transactions](#upsert_trd)<br>
8. [Create new transaction types to capture cash versus stock versus comissions](#trd_movements)<br>
9. [Create holdings report](#holdings)<br>
10. [View and track your comissions](#com_report)<br>


## Setup client and scope  <a id="setup"></a>  

Import lusid and non-lusid packages, validate client and define scope.

In [1]:
# Import the general purpose Python packages

import copy
import os

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools import cocoon as lpt
from lusidtools.cocoon.cocoon_printer import *
import lusid
import lusid.models as models
import lusid_sample_data as import_data
import globalfund as global_fund_tools

# Import data wrangling packages
# One we havent used in other notebooks is the flatten_json package
# We use this to flatten the results of a get_holdings response to parse the data into a neat DataFrame

from IPython.display import display_html
import pandas as pd
from pandas.io.json import json_normalize
from flatten_json import flatten
import json
pd.set_option('display.max_columns', None)

# Initiate an API Factory which is the client side object for interacting with LUSID APIs

api_factory = import_data.authenticate_secrets()

In [2]:
# define a scope and a porfolio to to used in this notebook

scope = "custodian"
portfolio = "LUSID_COM001"

## Read in transactions data from file <a id="trd_csv_read"></a>  

First we need to load some trades into a pandas DataFrame from a trades CSV file. For the purposes of this example, we use 20 hypothetical trades across 10 seperate FTSE100 equities. The CSV file uses unknown column headers from the custodian's internal system.

In [4]:
# The custodian file contains a "portfolio code" value
# For the purposes of this notebook, we also allow users of the notebook to pass their own "portfolio" value

cust_trades_df = pd.read_csv("data/trade-comms/custodian_trades.csv")
cust_trades_df["fund_code"] = portfolio
cust_trades_df.head(5)

,fund_code,fund_name,isin,client_id,security_description,security_group,trd_number,transaction_code,trade_date,settle_date,units,price,principal,trading_currency,country_of_risk,country_of_risk_name,trade_tax,broker_commission,exchange_rate
0,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB0002162385,EQ_1234,Aviva,COMMON STOCK,trd_123,custodian_buy,27/11/2019,29/11/2019,124626,4.01,499999.51,GBP,GB,United Kingdom,350,20,1
1,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB0002162385,EQ_1234,Aviva,COMMON STOCK,trd_124,custodian_buy,27/11/2019,29/11/2019,12462,4.01,49997.54,GBP,GB,United Kingdom,350,30,1
2,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB0000566504,EQ_1235,BHP,COMMON STOCK,trd_125,custodian_buy,27/11/2019,29/11/2019,57777,17.31,1000004.32,GBP,GB,United Kingdom,360,20,1
3,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB0000566504,EQ_1235,BHP,COMMON STOCK,trd_126,custodian_buy,27/11/2019,29/11/2019,57777,17.31,1000004.32,GBP,GB,United Kingdom,360,40,1
4,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB0031348658,EQ_1236,Barclays,COMMON STOCK,trd_127,custodian_buy,27/11/2019,29/11/2019,143119,1.75,250000.27,GBP,GB,United Kingdom,355,20,1


## Set up mapping <a id="trd_map"></a>  

Next we setup the mapping from required LUSID fields to the corresponding columns in the dataframe (https://www.lusid.com/docs/api/).<br>

In the most basic sense, the mapping is a mapping between what LUSID and the external world understands to be a specific field.

In [5]:
mapping = {
    
    "cash_flag": {
        "cash_identifiers": {
            "security_group" : ["CASH"]
        },
        "implicit": "trading_currency"
    },
    "instruments":{
        "identifier_mapping": {
            "ClientInternal": "client_id",
            "Isin": "isin"
        },
        "required": {
            "name": "security_description"
        },
        "properties":[
            "security_group",
            "country_of_risk",
            "country_of_risk_name",
        ]
    },
    "transactions": {
        "identifier_mapping": {
            "ClientInternal": "client_id",

        },
        "required": {
            "code": "fund_code",
            "transaction_id": "trd_number",
            "type": "transaction_code",
            "transaction_price.price": "price",
            "transaction_price.type": "$Price",
            "total_consideration.amount": "principal",
            "units": "units",
            "transaction_date": "trade_date",
            "total_consideration.currency": "trading_currency",
            "settlement_date": "settle_date"
        },
        "optional": {
            "source": "$lusid_security_services"
        },
        "properties": [
            "exchange_rate",
            "trade_tax",
            "broker_commission",
        ]
    },
    "portfolios": {
        "required": {
            "code": "fund_code",
            "display_name": "fund_name",
            "base_currency": "$GBP",
            "created": "$2018-01-01T00:00:00+00:00"
        }
    },
}


## Set up the instrument master <a id="trd_im"></a>  

Next we setup the instrument master. For more details on Instruments in LUSID, please see our detailed and specific notebook on the topic [HERE](https://github.com/finbourne/sample-notebooks/blob/master/examples/use-cases/Instruments.ipynb).

In [6]:
# Create an instruments specific DataFrame for uploading

cust_trades_df_inst, mapping = lpt.identify_cash_items(cust_trades_df, mapping, remove_cash_items=True, file_type="instruments")
cust_trades_df_inst.tail(2)

,fund_code,fund_name,isin,client_id,security_description,security_group,trd_number,transaction_code,trade_date,settle_date,units,price,principal,trading_currency,country_of_risk,country_of_risk_name,trade_tax,broker_commission,exchange_rate,__currency_identifier_for_LUSID
18,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB00B1XZS820,EQ_1243,Anglo American plc,COMMON STOCK,trd_141,custodian_buy,27/11/2019,29/11/2019,36452,20.57,749999.90,GBP,GB,United Kingdom,355,20,1,None
19,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB00B1XZS820,EQ_1243,Anglo American plc,COMMON STOCK,trd_142,custodian_buy,27/11/2019,29/11/2019,36452,20.57,749999.90,GBP,GB,United Kingdom,355,40,1,None


In [7]:
# Upsert the instruments into LUSID

result = lpt.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=cust_trades_df_inst,
    mapping_required=mapping["instruments"]["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=mapping["instruments"]["identifier_mapping"],
    property_columns=mapping["instruments"]["properties"],
    properties_scope=scope
)

succ, failed, errors = format_instruments_response(result)
print(f"number of successful upserts: {len(succ)}")
print(f"number of failed upserts    : {len(failed)}")
print(f"number of errors            : {len(errors)}")

number of successful upserts: 10
number of failed upserts    : 0
number of errors            : 0


Once the instruments have been upserted, we now have an instance of LUSID with an instrument master:

![Init](img/trade-comms/lusid_instruments.PNG)

## Create the property definition  <a id="prop_def"></a>  

Next we create a property definition which will allow us to split out a holding into a number of sub-holdings (such as comission).<br>

Every properties consist of a "{domain}/{scope}/{code}" configuration. This allows us to organise the data for easy access later. In the example below, we want to track asset type in the custodian scope on the transaction. Therefore the property is "Transaction/custodian/AssetType".

In [8]:
try:

    api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(

        lusid.models.CreatePropertyDefinitionRequest(

            domain = "Transaction",
            scope = "custodian",
            code = "AssetType",
            value_required = None,
            display_name = "Type of asset movement",
            data_type_id = lusid.ResourceId(scope="system", code="string"),
            life_time = None

        )
    )
    
    print("Property created.")

except:
    
    print("API Exception: Property might already exist")

API Exception: Property might already exist


## Create the portfolio <a id="trd_port"></a>  

Then we create a portfolio with the new property as a sub-holding key. These keys are used to organise a portfolio's holding from a set of transactions. 

https://support.finbourne.com/what-are-subholding-keys

In [9]:
result = lpt.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=cust_trades_df,
    mapping_required=mapping["portfolios"]["required"],
    mapping_optional={},
    file_type="portfolios",
    sub_holding_keys=["Transaction/custodian/AssetType"]
)

succ, failed = format_portfolios_response(result)
print(f"number of successful portfolios requests: {len(succ)}")
print(f"number of failed portfolios requests    : {len(failed)}")

number of successful portfolios requests: 1
number of failed portfolios requests    : 0


Now we have an instrument master with a portfolio in LUSID.

![Init](img/trade-comms/lusid_port.PNG)

## Upsert your transactions <a id="upsert_trd"></a>  

Now we can upsert our transactions file to LUSID.

In [10]:
cust_trades_df, mapping = lpt.identify_cash_items(cust_trades_df, mapping, remove_cash_items=False, file_type="transactions")
cust_trades_df.tail(2)

,fund_code,fund_name,isin,client_id,security_description,security_group,trd_number,transaction_code,trade_date,settle_date,units,price,principal,trading_currency,country_of_risk,country_of_risk_name,trade_tax,broker_commission,exchange_rate,__currency_identifier_for_LUSID,LUSID.base_currency,LUSID.created
19,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB00B1XZS820,EQ_1243,Anglo American plc,COMMON STOCK,trd_142,custodian_buy,27/11/2019,29/11/2019,36452,20.57,749999.90,GBP,GB,United Kingdom,355,40,1,None,GBP,2018-01-01T00:00:00+00:00
20,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,NaN,NaN,GBP Cash,CASH,trd_143,custodian_sub,27/11/2019,29/11/2019,12000000,1.00,12000000.00,GBP,GB,United Kingdom,0,0,1,GBP,GBP,2018-01-01T00:00:00+00:00


In [11]:
result = lpt.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=cust_trades_df,
    mapping_required=mapping["transactions"]["required"],
    mapping_optional=mapping["transactions"]["optional"],
    file_type="transactions",
    identifier_mapping=mapping["transactions"]["identifier_mapping"],
    property_columns=mapping["transactions"]["properties"],
    properties_scope=scope
)

succ, failed = format_transactions_response(result)
print(f"number of successful portfolios requests: {len(succ)}")
print(f"number of failed portfolios requests    : {len(failed)}")

number of successful portfolios requests: 1
number of failed portfolios requests    : 0


## Map the transaction types to movements <a id="trd_movements"></a>  

Next we configure some "movements" which tells LUSID how to build transactions into holdings

In [12]:
# First, let us configure a "side" called TradeCommission.
# In most simple terms, this side is used to capture the broker commission 
# from the "broker_commission" property on a Transaction.
# Recall also that we created that property from the initial CSV file.

try:

    api_factory.build(lusid.api.SystemConfigurationApi).create_side_definition(
        side=lusid.models.SideConfigurationDataRequest(
            side="TradeCommission",
            security="Txn:SettleCcy",
            currency="Txn:SettlementCurrency",
            rate="Transaction/custodian/exchange_rate",
            units="Transaction/custodian/broker_commission",
            amount="Transaction/custodian/broker_commission"
        )
    )

except:
    
    print("Creation of side failed")

In [13]:
# Then we create some "transaction type" configurations.
# These are instructions for LUSID when building holdings from transactions.
# For example, when LUSID sees that a transaction has a "transaction type" of "custodian_buy", it knows it needs to do
# three things:
# (1) Increase the stock in the portfolio 
# (2) Decrease the cash
# (3) Increase the commission paid


response = global_fund_tools.create_transaction_type_configuration(
    api_factory=api_factory, 
    aliases=[
        ("custodian_buy", "lusid_security_services")
    ],
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types='StockMovement',
            side='Side1',
            direction=1,
            properties=None,
            mappings=[
            models.TransactionPropertyMappingRequest(
            property_key="Transaction/custodian/AssetType",
                    set_to="Stock"
                )]),
            models.TransactionConfigurationMovementDataRequest(
            movement_types='CashCommitment',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=[
            models.TransactionPropertyMappingRequest(
            property_key="Transaction/custodian/AssetType",
                    set_to="Cash"
                )]),
            models.TransactionConfigurationMovementDataRequest(
            movement_types='CashCommitment',
            side='TradeCommission',
            direction=-1,
            properties=None,
            mappings=[
                models.TransactionPropertyMappingRequest(
                    property_key="Transaction/custodian/AssetType",
                    set_to="Commission"
                )])])

In [14]:
response = global_fund_tools.create_transaction_type_configuration(
    api_factory=api_factory, 
    aliases=[
        ("custodian_sub", "lusid_security_services")
    ],
    movements=[
            models.TransactionConfigurationMovementDataRequest(
            movement_types='CashAccrual',
            side='Side1',
            direction=1,
            properties=None,
            mappings=[
            models.TransactionPropertyMappingRequest(
            property_key="Transaction/custodian/AssetType",
                    set_to="Cash")


                ])])

## Create holdings report <a id="holdings"></a>  

Finally we generate a holdings report in LUSID using the get_holdings endpoint. Based on the configuration above, a number of things happen when we call this endpoint. It's worth going through them one-by-one:
<br>

* First, LUSID gathers the transactions for this portfolio from the events register (these are the transactions which will be used to build holdings).
* Then based on the movements configuration defined above, a transaction will then be used to build a holding as follows.<br>
* The transactions with a transaction type of "custodian_buy" will increase holdings, decrease cash, and increase the amount of commission paid.
* The transactions with a transaction type of "custodian_sub" will increase the cash only. This is designed to represent a portfolio subscription.
* In reality there will also be an inverse movement configured for "custodian_buy" and "custodian_sub" (i.e. "custodian_sell" and "custodian_red")

In [25]:
def generate_holdings_df(scope, portfolio):
    
    holdings_response = api_factory.build(
        lusid.api.transaction_portfolios_api.TransactionPortfoliosApi).get_holdings(scope, portfolio)
    
    holdings_df = pd.DataFrame(flatten(trade.to_dict(), '.') for trade in  holdings_response.values)
    
    holdings_df["name"] = holdings_df["instrument_uid"].apply(
    lambda x: api_factory.build(lusid.api.InstrumentsApi).get_instrument(
        identifier_type="LusidInstrumentId",
        identifier=x).name)
    
    holdings_df = holdings_df.rename(columns = {"sub_holding_keys.Transaction/custodian/AssetType.value.label_value": "AssetType",
                             "cost.amount": "Value"})
    
    return holdings_df[["AssetType","name", "Value"]]
    
    

In [31]:
holdings_df = generate_holdings_df(scope, portfolio)

In [32]:
holdings_df_formatted = holdings_df.copy()
holdings_df_formatted["Value"] = holdings_df_formatted["Value"].map('£{:,.2f}'.format)
holdings_df_formatted

,AssetType,name,Value
0,Stock,Aviva,"£549,997.05"
1,Stock,BHP,"£2,000,008.64"
2,Stock,Barclays,"£500,000.54"
3,Stock,BP,"£1,000,002.74"
4,Stock,HSBC,"£249,996.80"
5,Stock,Morrisons,"£749,998.42"
6,Stock,Tesco,"£500,001.84"
7,Stock,Rightmove,"£1,000,002.02"
8,Stock,vodafone,"£1,500,001.54"
9,Stock,Anglo American plc,"£1,499,999.80"


## View and track your comissions <a id="com_report"></a>  

We can then subtotal the results.

Here we can see that the portfolio has paid £500 in commission over its lifetime.

In [33]:
holdings_df[["AssetType","Value"]].groupby("AssetType").sum()

,Value
AssetType,
Cash,2449990.61
Commission,-500.00
Stock,9550009.39
